# **LL(1) Parser**

An LL(1) parser is a type of recursive descent parser that can parse a context-free grammar (CFG) in linear time. It is called an LL(1) parser because it reads the input from left to right (L), uses a leftmost derivation (L), and looks one symbol ahead (1).

## Assumptions Used

##### 1) Grammar rules are stored as a dictionary in which each non-terminal is a key and its value is a lsit of this non-terminal's rules as follows:
&ensp; **Given the Following Grammar:** <br>
&ensp; productionRules = { <br>
&emsp; "E": ["E+T", "T", "Ta"], <br>
&emsp; "T": ["T*F", "F"], <br>
&emsp; "F": ["(E)", "id"] <br>
##### 2) We replaced any non-terminal that is followed by a (') with a random char for the ease of reading the rules
&ensp; (i.e.) E' is converted to A
##### 3) # means epsilon 
##### 4) if the user didn't define the start symbol, we will take the first key in the dictionary
##### 5) if the user didn't define the list of terminals and non-terminals, we will proceed it as the capital letters are non-terminals and the rest are terminals
&ensp; (i.e.) E is a non-terminal
&ensp; (i.e.) e is a terminal

## Stages of LL1 Parsing

### 1. Left Recursion & Left Factoring Removal

##### a] Define the Grammar Rules, Start Symbol and list of terminals

In [ ]:
start_symbol = 'E'

productionRules = {
    "E'": ["+TE'", "#"],
    "E": ["TE'"],
    "T'": ["*FT'", "#"],
    "T": ["FT'"],
    "F": ["(E)", "id"]
}


# productionRules = {
#     "E": ["E+T", "T"],
#     "T": ["T*F", "F"],
#     "F": ["(E)", "id"]
# }


In [ ]:
# terminal= set()
# non_t= set()
# for key in productionRulesV3:
#     for rule in productionRulesV3[key]:
#         for char in rule:
#             if (char.isupper()):
#                 non_t.add(char)
#             else:
#                 if (char !="#"):
#                     terminal.add(char)
                
                
                
# term_userdef=list(terminal)
# nonterm_userdef=list(terminal)
                
term_userdef =['id', '(', '+', '*', ')','$']

In [ ]:
# Find the start symbol
if start_symbol == None:
  start_symbol = next(iter(productionRules))
start_symbol 

'E'

##### b] Import Needed Libraries

In [ ]:
#Imports
import random
import string

##### c] (optional) Remove all ' (ie. E' ==> any other random char)

In [ ]:
# A Function to generate a random capital character that is not used in either of the dictionaries
def generateKey(originalDictionary, newDictionary):
    # Generate a list of uppercase letters
    uppercase_letters = list(string.ascii_uppercase)
    
    # Return a random uppercase letter
    return random.choice(uppercase_letters)
    
    if (randomChar not in originalDictionary) and (randomChar not in newDictionary) :
        if randomChar == None:
            generateKey(originalDictionary, newDictionary)
        else:
            return randomChar
    else:
        generateKey(originalDictionary, newDictionary)
    

In [ ]:
# A Function to update the rule list of each key according to the new generated key

# Create a dictionary to store the previous replacements (if the value to be replace was replaced in a previous rule)
previous_replacements = {}

def updateRuleList (new_key, rule_list):
    
    # Iterate over the rules list given
    for i, rule in enumerate(rule_list):
        # Iterate over the characters in the rule
        for j, ruleChar in enumerate(rule):
            # Check if the character is a "'"
            if ruleChar == "'":
                # Replace the character with the new key
                rule_list[i] = rule[:j] + new_key + rule[j+1:]
                # Remove the previous character to the "'"
                rule_list[i] = rule_list[i][:j-1] + rule_list[i][j:]
                    
                # Check if the previous character was previously replaced
                if rule[j-1] in previous_replacements:
                    # Replace the previous character with the old one
                    rule_list[i] = rule_list[i][:j-1] + previous_replacements[rule[j-1]] + rule_list[i][j:]
                    
                # Update the dictionary with the current replacement
                previous_replacements[rule[j-1]] = new_key
                
    # Return the modified dictionary
    return rule_list

In [ ]:
productionRulesV1 = {}

# Iterate over the keys in the dictionary
for key in productionRules:
    # Generate new random key
    newKey = generateKey (productionRules, productionRulesV1)
    
    flag2=0
    # Iterate over the characters of the key
    for char2 in key:
        if char2 == "'":
            flag2=1
            
    if flag2:
        previous_replacements[key] = newKey
        productionRulesV1[newKey] = updateRuleList(newKey, productionRules[key])
    else:
        productionRulesV1[key] = updateRuleList(newKey, productionRules[key])


In [ ]:
# First updated version of the production rules
productionRulesV1

{'J': ['+TJ', '#'],
 'E': ['TJ'],
 'V': ['*FV', '#'],
 'T': ['FV'],
 'F': ['(E)', 'id']}

In [ ]:
def remove_left_recursion(production_rules, new_production_rules):
    rulesCount=0
    # Iterate over the keys in the rules map
    for key in production_rules:
        # Iterate over the rules in the list
        for i, rule in enumerate(production_rules[key]):
            rulesCount+=1
            # Check if the key is the first character of the rule
            if key == rule[0]:
                    print("Rule ",rulesCount,": Is left recurcive")
#                         "E": ["E+T", "T"],

#                         TO:

#                         "E": ["T S"],
#                         "S": ["+ T S", "#"],

                    # Create a new non-terminal symbol (Generate a random capital character that is not used in either of the dictionaries)
                    newKey = generateKey(new_production_rules, production_rules)

                    # Initialize the new rule list
                    newRuleList = []
                    originalRuleList = []

                    # Iterate over the rules in the list
                    for rule in production_rules[key]:
                        # If the rule starts with the key, add it to the new rule list with the new key
                        if rule[0] == key:
                            newRuleList.append(rule[1:] + newKey)
                        # Otherwise, keep it in the original rule list with the same original key but with right recursion ==>  + newKey
                        else:
                            originalRuleList.append(rule + newKey)
                    newRuleList.append("#")
                    # Add the newRuleList with the newKey to the new_production_rules dictionary
                    new_production_rules[newKey] = newRuleList
                    # Update the originalRuleList in the new_production_rules dictionary
                    new_production_rules[key] = originalRuleList
            else:
                # Key thant doesn't have left recursion 
                if key not in new_production_rules:
                    originalRuleList = []
                    # Iterate over the rules in the list
                    for rule in production_rules[key]:
                        originalRuleList.append(rule)
                    new_production_rules[key] = originalRuleList
    return new_production_rules


In [ ]:
def remove_left_factoring(production_rules, new_production_rules):                  
    rulesCount=0                
    # Iterate over the keys in the rules map
    for key in production_rules:
        # Iterate over the rules in the list
        for i, rule in enumerate(production_rules[key]):
            rulesCount+=1
            # Check if there is a rule in the current list starting with the same character as the current rule
            found = False
            for j, s in enumerate(production_rules[key]):
                if i != j and s[0] == rule[0]:
                    found = True
                    break
            if found:
                print("Rule ",rulesCount,": Has Left factoring")
                
#                          "E": ["T", "Ta"]

#                         TO:

#                         "E": ["TY"],
#                         "Y" : ["#","a"]

                # Create a new non-terminal symbol (Generate a random capital character that is not used in either of the dictionaries)
                newKey = generateKey(new_production_rules, production_rules)

                # Initialize the new rule list
                newRuleList = []
                originalRuleList = []

                # Iterate over the rules in the list
                for rule in production_rules[key]:
                    # If there is a rule in the current list starting with the same character as the current rule
                    for j, s in enumerate(production_rules[key]):
                        if i != j and s[0] == rule[0]:
                            newRuleList.append(rule[1:])
                            newRuleList.append(s[1:])
                            
                            if s[1:]=="":
                                newRuleList.append('#')
                                newRuleList.remove('')
                            elif rule[1:]=="":
                                newRuleList.append('#')
                                newRuleList.remove('')
                                
                            
                            originalRuleList.append(rule[0]+newKey)
                            # Remove both rules from the list
                            production_rules[key].remove(rule)
                            production_rules[key].remove(s)
                            
                        elif i != j and s[0] != rule[0]:
                            originalRuleList.append(s)
                            
                        # Add the newRuleList with the newKey to the new_production_rules dictionary
                        new_production_rules[newKey] = newRuleList
                        # Update the originalRuleList in the new_production_rules dictionary
                        new_production_rules[key] = originalRuleList
        else:
            # Key thant doesn't have left factoring
            if key not in new_production_rules:
                originalRuleList = []
                # Iterate over the rules in the list
                for rule in production_rules[key]:
                    originalRuleList.append(rule)
                new_production_rules[key] = originalRuleList
                                               

    return new_production_rules
            

In [ ]:
# Second updated version of the production rules
productionRulesV2 = {}

remove_left_recursion(productionRulesV1, productionRulesV2);
productionRulesV2

{'J': ['+TJ', '#'],
 'E': ['TJ'],
 'V': ['*FV', '#'],
 'T': ['FV'],
 'F': ['(E)', 'id']}

In [ ]:
# Third updated version of the production rules
productionRulesV3 = {}

remove_left_factoring(productionRulesV2, productionRulesV3);
productionRulesV3

{'J': ['+TJ', '#'],
 'E': ['TJ'],
 'V': ['*FV', '#'],
 'T': ['FV'],
 'F': ['(E)', 'id']}

### 2. Computing FIRST Sets

In [ ]:
first_set={}

def comp_first (val):
    if(val.split()[0][0] not in productionRulesV3):
        if val.islower():
            return [val]
        else:
            return [val[0]]
    first=[]
    st = val.split()
    for value in productionRulesV3[val[0]]:  
        if(value.islower()==True):
            first.append(value)
        elif(st[0]!=value.split()[0]):
            first.extend(comp_first(value.split()[0][0]))          
    return  first  

def comp_first_set (production_rule):
    for key in production_rule:
        first_set[key] = comp_first(key)
    return first_set

comp_first_set(productionRulesV3)
first_set

{'J': ['+', '#'],
 'E': ['(', 'id'],
 'V': ['*', '#'],
 'T': ['(', 'id'],
 'F': ['(', 'id']}

### 3. Computing FOLLOW Sets

In [ ]:
def FollowSet(nt):
    res=[]
    ansSet= set()
    if nt == start_symbol:
        ansSet.add('$')
        
    for item in productionRulesV3:
        rh= productionRulesV3[item]
        for subrule in rh:
            while nt in subrule:
                index_nt = subrule.index(nt)
                subrule = subrule[index_nt + 1:]
                ####first role of the follow 
                if len(subrule) != 0: 
                    res = comp_first(subrule)
                    #### second role of follow aBb
                    if '#' in res:
                        tempList = []
                        res.remove('#')
                        ansNew = FollowSet(item)
                        if ansNew != None:
                            if type(ansNew) is list:
                                tempList = res + ansNew
                            else:
                                tempList = res + [ansNew]
                        else:
                                tempList = res
                        res = tempList
                else:
                    if nt!= item:
                        res = FollowSet(item)
                    
                if res is not None:
                    if type(res) is list:
                        for i in res:
                            ansSet.add(i)
                    else:
                        ansSet.add(res)
    return list(ansSet)

In [ ]:
follow_set={}

def FollowSetCompute():
    for elemnt in productionRulesV3:
        ansSet= set()
        ans= FollowSet(elemnt)
        if ans is not None:
            for a in ans:
                ansSet.add(a)
        follow_set[elemnt]= ansSet
    print("Follow sets :")
    key_list= list(follow_set.keys())
    idx=0

    for i in follow_set:
        print(f"Follow({key_list[idx]})" f" = {follow_set[i]}")
        idx+=1

In [ ]:
FollowSetCompute()

Follow sets :
Follow(J) = {')', '$'}
Follow(E) = {')', '$'}
Follow(V) = {'+', ')', '$'}
Follow(T) = {'+', ')', '$'}
Follow(F) = {'+', '*', ')', '$'}


### 4. Generating Parsing Table

In [ ]:
def parsingTableFunc(production_rule, firstDic, followsDic):
    import copy
    print("\nFirsts and Follow table\n")
    lenfirst = 0
    lenfollow = 0
    for i in production_rule:
        l1 = len(str(firstDic[i]))
        l2 = len(str(followsDic[i]))
        if l1 > lenfirst:
            lenfirst = l1
        if l2 > lenfollow:
            lenfollow = l2
    print(f"{{:<{10}}} "
        f"{{:<{lenfirst + 5}}} "
        f"{{:<{lenfollow + 5}}}"
        .format("Non-Terminal", "FIRST", "FOLLOW"))
    for i in production_rule:
        print(f"{{:<{10}}} "
            f"{{:<{lenfirst + 5}}} "
            f"{{:<{lenfollow + 5}}}"
            .format(i, str(firstDic[i]), str(followsDic[i])))
    nonterminals = list(production_rule.keys())
    terminals = copy.deepcopy(term_userdef)
    terminals.append('$')
    matrixx = []
    for x in production_rule:
        row = []
        for y in terminals:
            row.append('')
        matrixx.append(row)
    grammar_is_LL = True
    for lhs in production_rule:
        rhs = production_rule[lhs]
        for y in rhs:
            res = comp_first(y)
            if '#' in res:
                res.remove('#')
                res = list(res) + list(followsDic[lhs])
            ttemp = []
            for c in res:
                xnt = nonterminals.index(lhs)
                yt = terminals.index(c)
                if matrixx[xnt][yt] == '':
                    matrixx[xnt][yt] = matrixx[xnt][yt]  + f"{lhs}->{''.join(y)}"
                else:
                    if f"{lhs}->{y}" in matrixx[xnt][yt]:
                        continue
                    else:
                        grammar_is_LL = False
                        matrixx[xnt][yt] = matrixx[xnt][yt] + f",{lhs}->{''.join(y)}"
    print("\nParsing Table:\n")
    formatt = "{:>12}" * len(terminals)
    print(formatt.format(*terminals))
    j = 0
    for y in matrixx:
        formatt1 = "{:>12}" * len(y)
        print(f"{nonterminals[j]} {formatt1.format(*y)}")
        j += 1
    return (matrixx, grammar_is_LL, terminals)

In [ ]:
(parsing_table, result, tabTerm) = parsingTableFunc(productionRulesV3, first_set, follow_set)


Firsts and Follow table

Non-Terminal FIRST            FOLLOW                   
J          ['+', '#']       {')', '$'}               
E          ['(', 'id']      {')', '$'}               
V          ['*', '#']       {'+', ')', '$'}          
T          ['(', 'id']      {'+', ')', '$'}          
F          ['(', 'id']      {'+', '*', ')', '$'}     

Parsing Table:

          id           (           +           *           )           $           $
J                               J->+TJ                    J->#        J->#            
E        E->TJ       E->TJ                                                            
V                                 V->#      V->*FV        V->#        V->#            
T        T->FV       T->FV                                                            
F        F->id      F->(E)                                                            


### 5. Parsing Inputs

In [ ]:
def check(input_string):
    stack = [start_symbol, '$']
    input_str = []
    input_string.strip()
    input_string = input_string.split()
    input_string.reverse()
    input_str=['$']+input_string
    while True:
        if stack == ['$'] and input_str == ['$']:
            print("{:>20} {:>20} {:>25}".
                      format(' '.join(input_str),
                             ' '.join(stack),
                             f"T[{stack[0]}][{input_str[-1]}] = {'$'}"))
            print("Valid String")
            return
        elif stack[0].isupper():
            if (input_str[-1] not in term_userdef):
                print("Invalid String")
                return
            x = list(productionRulesV3.keys()).index(stack[0])
            y = term_userdef.index(input_str[-1])
            if parsing_table[x][y] != '':
                entry=parsing_table[x][y]
                newstring = entry.split("->")
                print("{:>20} {:>20} {:>25}".
                      format(' '.join(input_str),
                             ' '.join(stack),
                             f"T[{stack[0]}][{input_str[-1]}] = {entry}"))
                newstring[1]=newstring[1].replace('#', '').strip()
                st=[]
                if newstring[1] not in term_userdef:
                    for i in range(len(newstring[1])):
                        st.append(newstring[1][i])
                    st.extend(stack[1:])
                    stack=st
                else:
                    if(len(newstring[1])>0):
                        st=[newstring[1]]
                    st.extend(stack[1:])
                    stack=st
            else:
                print("Invalid String")
                return
        else:
            if stack[0] == input_str[-1]:
                print("{:>20} {:>20} {:>25}".
                      format(' '.join(input_str),
                             ' '.join(stack),
                             f"T[{stack[0]}][{input_str[-1]}] = {str(input_str[-1])}"))
                input_str = input_str[:-1]
                stack = stack[1:]
            else:
                print("invalid")
                return

In [ ]:
check("id + id * id + id")

 $ id + id * id + id                  E $          T[E][id] = E->TJ
 $ id + id * id + id                T J $          T[T][id] = T->FV
 $ id + id * id + id              F V J $          T[F][id] = F->id
 $ id + id * id + id             id V J $            T[id][id] = id
    $ id + id * id +                V J $            T[V][+] = V->#
    $ id + id * id +                  J $          T[J][+] = J->+TJ
    $ id + id * id +              + T J $               T[+][+] = +
      $ id + id * id                T J $          T[T][id] = T->FV
      $ id + id * id              F V J $          T[F][id] = F->id
      $ id + id * id             id V J $            T[id][id] = id
         $ id + id *                V J $          T[V][*] = V->*FV
         $ id + id *            * F V J $               T[*][*] = *
           $ id + id              F V J $          T[F][id] = F->id
           $ id + id             id V J $            T[id][id] = id
              $ id +                V J $       